# Training YOLOv3 on Fabric Stain Dataset

This notebook demonstrates how to train a YOLOv3 model on a custom dataset of fabric stains. The dataset consists of images stored in `.\\fabric_stain_dataset/split_images` and annotations in `.\\fabric_stain_dataset/annotations`.

In [2]:
# Install necessary libraries
!pip install tensorflow
!pip install opencv-python
!pip install lxml
!pip install matplotlib
!pip install PyYAML
!pip install tqdm
!pip install absl-py
!pip install tensorflow-addons


[notice] A new release of pip is available: 24.0 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 24.0 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip



   ---------------------------------------- 0.0/3.8 MB ? eta -:--:--
   ---------------------------------------- 0.0/3.8 MB ? eta -:--:--
   ---------------------------------------- 0.0/3.8 MB 653.6 kB/s eta 0:00:06
   --- ------------------------------------ 0.3/3.8 MB 2.7 MB/s eta 0:00:02
   ------ --------------------------------- 0.6/3.8 MB 3.6 MB/s eta 0:00:01
   --------- ------------------------------ 0.9/3.8 MB 4.7 MB/s eta 0:00:01
   ------------- -------------------------- 1.3/3.8 MB 5.3 MB/s eta 0:00:01
   ------------------ --------------------- 1.7/3.8 MB 6.1 MB/s eta 0:00:01
   ---------------------- ----------------- 2.2/3.8 MB 6.3 MB/s eta 0:00:01
   ---------------------------- ----------- 2.7/3.8 MB 6.8 MB/s eta 0:00:01
   ---------------------------------- ----- 3.3/3.8 MB 7.7 MB/s eta 0:00:01
   ---------------------------------------  3.8/3.8 MB 8.1 MB/s eta 0:00:01
   ---------------------------------------- 3.8/3.8 MB 7.8 MB/s eta 0:00:00



[notice] A new release of pip is available: 24.0 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 24.0 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 24.0 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


     ---------------------------------------- 0.0/57.7 kB ? eta -:--:--
     -------------------- ----------------- 30.7/57.7 kB 660.6 kB/s eta 0:00:01
     -------------------------------------- 57.7/57.7 kB 765.1 kB/s eta 0:00:00
   ---------------------------------------- 0.0/78.5 kB ? eta -:--:--
   ---------------------------------------- 78.5/78.5 kB 2.2 MB/s eta 0:00:00



[notice] A new release of pip is available: 24.0 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 24.0 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip
ERROR: Could not find a version that satisfies the requirement tensorflow-addons (from versions: none)
ERROR: No matching distribution found for tensorflow-addons

[notice] A new release of pip is available: 24.0 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


## Import Libraries

In [4]:
import tensorflow as tf
from keras import layers, models
import numpy as np
import cv2
import os
import matplotlib.pyplot as plt
from tqdm import tqdm
import yaml
from absl import app, flags, logging
from absl.flags import FLAGS
from PIL import Image

KeyError: "Registering two gradient with name 'ReduceDataset'! (Previous registration was in register c:\\Users\\Vente\\Documents\\Code\\HumanComputerInteraction\\venv\\Lib\\site-packages\\tensorflow\\python\\framework\\registry.py:65)"

## Load Dataset

In [ ]:
import os

annotation_dir = os.path.join('fabric_stain_dataset', 'annotations', 'stain')
total_stains = 0
max_stains = 0

for annotation_file in os.listdir(annotation_dir):
    annotation_file = os.path.join(annotation_dir, annotation_file)
    with open(annotation_file, 'r') as file:
        num_stains = len(file.readlines())
        total_stains += num_stains
        if num_stains > max_stains:
            max_stains = num_stains

print("Total number of stains:", total_stains)
print("Maximum number of stains in a single image:", max_stains)

import numpy as np
import shutil

#Because there are multiple stains per iamge, this gets a bit tricky
source_dir = os.path.join('fabric_stain_dataset', 'images', 'stain')
train_dir = os.path.join('fabric_stain_dataset', 'split_images', 'train')
val_dir = os.path.join('fabric_stain_dataset', 'split_images', 'val')
train_annotation_dir = os.path.join('fabric_stain_dataset', 'split_annotations', 'train')
val_annotation_dir = os.path.join('fabric_stain_dataset', 'split_annotations', 'val')
annotation_dir = os.path.join('fabric_stain_dataset', 'annotations', 'stain')

# Create the balanced dataset folders if it doesn't exist
def handle_dir_creation(dir):
    if not os.path.exists(dir):
        os.makedirs(dir)
    else:
        # Remove all the files in the folder
        for file_name in os.listdir(dir):
            file_path = os.path.join(dir, file_name)
            try:
                if os.path.isfile(file_path):
                    os.unlink(file_path)
            except Exception as e:
                print(f"Failed to delete {file_path}. Reason: {e}")

# Create the directories if they don't exist
handle_dir_creation(train_dir)
handle_dir_creation(val_dir)
handle_dir_creation(train_annotation_dir)
handle_dir_creation(val_annotation_dir)

# Ensure the target directories exist
os.makedirs(train_dir, exist_ok=True)
os.makedirs(val_dir, exist_ok=True)
os.makedirs(train_annotation_dir, exist_ok=True)
os.makedirs(val_annotation_dir, exist_ok=True)


#We'll be using an 80-20 split for the training and validating datasets. this is pretty standard.
train_test_split = 0.8 

image_files = [f for f in os.listdir(source_dir) if os.path.isfile(os.path.join(source_dir, f))]

# Shuffle the list to ensure randomness
np.random.shuffle(image_files)

#Find the total number of stains in the dataset
total_stains = 0
for annotation_file in os.listdir(annotation_dir):
    annotation_file = os.path.join(annotation_dir, annotation_file)
    with open(annotation_file, 'r') as file:
        num_stains = len(file.readlines())
        total_stains += num_stains

num_train_stains = int(total_stains * 0.8)
current_train_stains = 0


for img in image_files:
    # Get the annotation file for the image
    annotation_file = os.path.join(annotation_dir, img.replace('.jpg', '.txt'))
    with open(annotation_file, 'r') as file:
        num_stains = len(file.readlines())
        if current_train_stains + num_stains <= num_train_stains:
            current_train_stains += num_stains
            shutil.copy(os.path.join(source_dir, img), os.path.join(train_dir, img))
            shutil.copy(annotation_file, os.path.join(train_annotation_dir, os.path.basename(annotation_file)))
        else:
            shutil.copy(os.path.join(source_dir, img), os.path.join(val_dir, img))
            shutil.copy(annotation_file, os.path.join(val_annotation_dir, os.path.basename(annotation_file)))

num_train_imgs = len(os.listdir(train_dir))
num_val_imgs = len(os.listdir(val_dir))

print(f"Moved {num_train_imgs} images with {current_train_stains} stains to {train_dir}")
print(f"Moved {num_val_imgs} images with {total_stains - current_train_stains} to {val_dir}")
print(f"Train val split is {current_train_stains / total_stains * 100:.2f}% - {(total_stains - current_train_stains) / total_stains * 100:.2f}%")

ValueError: setting an array element with a sequence. The requested array has an inhomogeneous shape after 1 dimensions. The detected shape was (398,) + inhomogeneous part.

## Define YOLOv3 Model

In [ ]:
def yolo_v3_model(input_shape):
    inputs = layers.Input(shape=input_shape)
    # Define the YOLOv3 architecture here
    # This is a simplified version for demonstration purposes
    x = layers.Conv2D(32, (3, 3), activation='relu')(inputs)
    x = layers.MaxPooling2D((2, 2))(x)
    x = layers.Conv2D(64, (3, 3), activation='relu')(x)
    x = layers.MaxPooling2D((2, 2))(x)
    x = layers.Conv2D(128, (3, 3), activation='relu')(x)
    x = layers.MaxPooling2D((2, 2))(x)
    x = layers.Flatten()(x)
    x = layers.Dense(512, activation='relu')(x)
    outputs = layers.Dense(2, activation='sigmoid')(x)
    model = models.Model(inputs, outputs)
    return model

input_shape = (32, 32, 1)  # Update input shape for 32x32 grayscale images
model = yolo_v3_model(input_shape)
model.summary()

Model: "functional_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer_1 (InputLayer)      │ (None, 32, 32, 1)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_3 (Conv2D)               │ (None, 30, 30, 32)     │           320 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_3 (MaxPooling2D)  │ (None, 15, 15, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_4 (Conv2D)               │ (None, 13, 13, 64)     │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_4 (MaxPooling2D)  │ (None, 6, 6, 64)       │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_5 (Conv2D)               │ (None, 4, 4, 128)      │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_5 (MaxPooling2D)  │ (None, 2, 2, 128)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_1 (Flatten)             │ (None, 512)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 512)            │       262,656 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ (None, 5)              │         2,565 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 357,893 (1.37 MB)

 Trainable params: 357,893 (1.37 MB)

 Non-trainable params: 0 (0.00 B)

## Compile and Train the Model

In [ ]:


# Assuming images and labels are preprocessed and ready for training
history = model.fit(images, labels, epochs=10, batch_size=8, validation_split=0.2)


# Directories
train_dir = os.path.join('fabric_stain_dataset', 'split_images', 'train')
val_dir = os.path.join('fabric_stain_dataset', 'split_images', 'val')
train_annotation_dir = os.path.join('fabric_stain_dataset', 'split_annotations', 'train')
val_annotation_dir = os.path.join('fabric_stain_dataset', 'split_annotations', 'val')

# Function to load annotations
def load_annotations(annotation_file):
    annotations = []
    with open(annotation_file, 'r') as file:
        for line in file:
            parts = line.strip().split()
            label = int(parts[0])
            x_center = float(parts[1])
            y_center = float(parts[2])
            width = float(parts[3])
            height = float(parts[4])
            annotations.append([label, x_center, y_center, width, height])
    return np.array(annotations)

# Function to preprocess data
def preprocess_data(image_dir, annotation_dir, image_size=(32, 32)):
    data = []
    annotations = []
    max_annotations = 0
    for img_name in os.listdir(image_dir):
        if img_name.endswith('.jpg'):
            image_path = os.path.join(image_dir, img_name)
            annotation_path = os.path.join(annotation_dir, img_name.replace('.jpg', '.txt'))
            
            if os.path.isfile(annotation_path):
                image = Image.open(image_path)
                image = image.resize(image_size)  # Resize image
                image = image.convert('L')  # Convert to grayscale
                image = np.array(image) / 255.0  # Normalize the image
                image = np.expand_dims(image, axis=-1)  # Add channel dimension
                annotation = load_annotations(annotation_path)
                
                data.append(image)
                annotations.append(annotation)
                
                # Update the maximum number of annotations
                if annotation.shape[0] > max_annotations:
                    max_annotations = annotation.shape[0]
    
    return np.array(data), annotations, max_annotations

# Preprocess training and validation data
train_data, train_annotations, train_max_annotations = preprocess_data(train_dir, train_annotation_dir)
val_data, val_annotations, val_max_annotations = preprocess_data(val_dir, val_annotation_dir)

# Determine the global maximum number of annotations
global_max_annotations = max(train_max_annotations, val_max_annotations)

# Pad annotations to the global maximum length
def pad_annotations(annotations, max_annotations):
    padded_annotations = []
    for annotation in annotations:
        if annotation.shape[0] < max_annotations:
            padding = np.zeros((max_annotations - annotation.shape[0], annotation.shape[1]))
            padded_annotation = np.vstack((annotation, padding))
        else:
            padded_annotation = annotation
        padded_annotations.append(padded_annotation)
    return np.array(padded_annotations)

train_annotations_padded = pad_annotations(train_annotations, global_max_annotations)
val_annotations_padded = pad_annotations(val_annotations, global_max_annotations)

# Flatten the annotations to match the model output
train_annotations_flat = train_annotations_padded.reshape((train_annotations_padded.shape[0], -1))
val_annotations_flat = val_annotations_padded.reshape((val_annotations_padded.shape[0], -1))

print("Shape of train_annotations_flat:", train_annotations_flat.shape)
print("Shape of val_annotations_flat:", val_annotations_flat.shape)

print("Data preprocessing completed")


model.compile(optimizer='adam',
              loss='mean_squared_error',  # Use appropriate loss function
              metrics=['accuracy'])
print("Model compiled")

# Print the model summary
print("Model Summary:")
model.summary()

# Data augmentation
datagen = ImageDataGenerator(
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

# Fit the model using the augmented data generator
print("Training the model with data augmentation")
history = model.fit(datagen.flow(train_data, train_annotations_flat, batch_size=32),
                    steps_per_epoch=len(train_data) // 32,
                    epochs=9,
                    validation_data=(val_data, val_annotations_flat))

# Print the training metrics
print("Training Metrics:")
for epoch in range(len(history.history['loss'])):
    print(f"Epoch {epoch + 1}:")
    print(f"  Loss: {history.history['loss'][epoch]}")
    print(f"  Accuracy: {history.history['accuracy'][epoch]}")
    print(f"  Validation Loss: {history.history['val_loss'][epoch]}")
    print(f"  Validation Accuracy: {history.history['val_accuracy'][epoch]}")

# Evaluate the model on the validation data
val_loss, val_accuracy = model.evaluate(val_data, val_annotations_flat, verbose=2)

# Print the validation metrics
print(f"Validation Loss: {val_loss}")
print(f"Validation Accuracy: {val_accuracy}")

# Predict on validation data
val_predictions = model.predict(val_data)

# Calculate Intersection over Union (IoU)
def calculate_iou(true_boxes, pred_boxes):
    ious = []
    for true_box, pred_box in zip(true_boxes, pred_boxes):
        _, x1_true, y1_true, w_true, h_true = true_box  # Unpack the true box
        _, x1_pred, y1_pred, w_pred, h_pred = pred_box  # Unpack the predicted box
        
        x1_true, y1_true = x1_true - w_true / 2, y1_true - h_true / 2
        x2_true, y2_true = x1_true + w_true, y1_true + h_true
        x1_pred, y1_pred = x1_pred - w_pred / 2, y1_pred - h_pred / 2
        x2_pred, y2_pred = x1_pred + w_pred, y1_pred + h_pred
        
        xi1, yi1 = max(x1_true, x1_pred), max(y1_true, y1_pred)
        xi2, yi2 = min(x2_true, x2_pred), min(y2_true, y2_pred)
        
        inter_area = max(0, xi2 - xi1) * max(0, yi2 - yi1)
        true_area = (x2_true - x1_true) * (y2_true - y1_true)
        pred_area = (x2_pred - x1_pred) * (y2_pred - y1_pred)
        
        union_area = true_area + pred_area - inter_area
        iou = inter_area / union_area if union_area != 0 else 0
        ious.append(iou)
    return np.mean(ious)

# Calculate IoU for validation data
ious = []
for i in range(len(val_annotations_flat)):
    true_boxes = val_annotations_flat[i].reshape(-1, 5)
    pred_boxes = val_predictions[i].reshape(-1, 5)
    iou = calculate_iou(true_boxes, pred_boxes)
    ious.append(iou)

mean_iou = np.mean(ious)
print(f"Mean IoU: {mean_iou}")

# Save the model
model.save('yolov3_cnn_model.h5')

ValueError: Argument `validation_split` is only supported for tensors or NumPy arrays.Found incompatible type in the input: [<class 'float'>, <class 'float'>, <class 'float'>, <class 'float'>, <class 'float'>, <class 'float'>, <class 'float'>, <class 'float'>, <class 'float'>, <class 'float'>, <class 'float'>, <class 'float'>, <class 'float'>, <class 'float'>, <class 'float'>, <class 'float'>, <class 'float'>, <class 'float'>, <class 'float'>, <class 'float'>, <class 'float'>, <class 'float'>, <class 'float'>, <class 'float'>, <class 'float'>, <class 'float'>, <class 'float'>, <class 'float'>, <class 'float'>, <class 'float'>, <class 'float'>, <class 'float'>, <class 'float'>, <class 'float'>, <class 'float'>, <class 'float'>, <class 'float'>, <class 'float'>, <class 'float'>, <class 'float'>, <class 'float'>, <class 'float'>, <class 'float'>, <class 'float'>, <class 'float'>, <class 'float'>, <class 'float'>, <class 'float'>, <class 'float'>, <class 'float'>, <class 'float'>, <class '

## Evaluate the Model

In [ ]:
loss, accuracy = model.evaluate(images, labels)
print(f'Loss: {loss}, Accuracy: {accuracy}')

## Visualize Training History

In [ ]:
plt.plot(history.history['accuracy'], label='accuracy')
plt.plot(history.history['val_accuracy'], label = 'val_accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.ylim([0, 1])
plt.legend(loc='lower right')
plt.show()